### Array Repeat

![](Images/95/95 Array Repeat.jpg)

### Use Case

![](Images/95/95 Use Case.jpg)

### Create Sample Delta Table

In [0]:
empData = [
    (111, 'Stephen', 'King', 2000),
    (222, 'Philip', 'Larkin', 8000),
    (333, 'John', 'Smith', 6000)
]

empSchema = ['Id', 'FirstName', 'LastName', 'Salary']

df = spark.createDataFrame(data=empData, schema=empSchema)
display(df)

In [0]:
from pyspark.sql.functions import explode, array_repeat, col

repeatDF = df.withColumn('Key_Col', array_repeat(col('Id'), 5))
display(repeatDF)

In [0]:
df1 = df.withColumn('Key_Col', explode(array_repeat(col('Id'), 10)))
display(df1)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, lit

win = Window().orderBy(lit('A'))
df2 = df1.withColumn('RowN', row_number().over(win))
df3 = df2.withColumn('NewID', col("Id") + col('RowN'))
display(df2)
display(df3)



In [0]:
outDF = df3.drop('Id', 'Key_Col', 'RowN').select(col("NewID").alias('Id'), 'FirstName', 'LastName', 'Salary')
display(outDF)

### UDF

In [0]:
def multiplyInputData(df, keyCol, multiplyFactor):
    from pyspark.sql.window import Window
    from pyspark.sql.functions import row_number, lit, explode, array_repeat
    df1 = df.withColumn(f'{keyCol}', explode(array_repeat(col(f'{keyCol}'), multiplyFactor)))
    win = Window().orderBy(lit('A'))
    df2 = df1.withColumn('RowN', row_number().over(win))
    df3 = df2.withColumn(f'{keyCol}', col('Id') + col('RowN')).drop('RowN')
    return df3
   

In [0]:
finalDF = multiplyInputData(df, 'Id', 10)
display(finalDF)